In [3]:
f = open('cifar-10-python/cifar-10-batches-py/batches.meta', 'r')

In [1]:
import tensorflow as tf

In [2]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' # 忽略低级别的警告

In [3]:
tf.test.is_gpu_available()

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


True

In [4]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [4]:
# 使用GPU
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
# The GPU id to use, usually either "0" or "1"
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

import keras
from keras.utils import np_utils
# from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense,Conv2D,MaxPooling2D,Dropout,Activation,Flatten,BatchNormalization
import keras
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
# config.gpu_options.allow_growth = True
config.gpu_options.per_process_gpu_memory_fraction = 0.9
session = InteractiveSession(config=config)

In [ ]:
import pickle

def unpickle(file):
      with open(file, 'rb') as fo:
              dict = pickle.load(fo, encoding='bytes')
      return dict

In [ ]:
# 训练集文件
train_files = ['data_batch_'+str(i) for i in range(1,6)]

train_data_list,train_label_list = [],[]

for f in train_files:
    
    fpath = 'cifar-10-python/cifar-10-batches-py/' + f
    batch_dict = unpickle(fpath)
    
    batch_data = batch_dict[b'data']
    batch_labels = batch_dict[b'labels']
    train_data_list.append(batch_data)
    train_label_list.append(batch_labels)

X_train = np.concatenate(train_data_list, axis = 0)
y_train = np.concatenate(train_label_list, axis = 0)

In [ ]:
# 测试集文件
test_batch = unpickle('cifar-10-python/cifar-10-batches-py/test_batch')

X_test = np.array(test_batch[b'data'])
y_test = np.array(test_batch[b'labels']) # list type

In [ ]:
label_names_batch = unpickle('cifar-10-python/cifar-10-batches-py/batches.meta')
label_names = label_names_batch[b'label_names']
label_names = [l.decode("utf-8") for l in label_names]

In [ ]:
print('训练集特征：',X_train.shape,'，训练集label',y_train.shape)
print('测试集特征：',X_test.shape,'，测试集label', y_test.shape)
print("类别名字：",label_names)

In [ ]:
num_classes = 10

In [ ]:
X_train = X_train.reshape(-1,3,32,32)
X_test = X_test.reshape(-1,3,32,32)

In [ ]:
fig = plt.figure(figsize=(8,3))

for i in range(num_classes):
    ax = fig.add_subplot(2, 5, 1 + i, xticks=[], yticks=[])
    idx = np.where(y_train[:]==i)[0]
    features_idx = X_train[idx,::]
    img_num = np.random.randint(features_idx.shape[0])
    im = np.transpose(features_idx[img_num,::], (1, 2, 0))
    ax.set_title(label_names[i])
    plt.imshow(im)
plt.show()

In [ ]:
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

# 归一化
X_train /= 255
X_test /= 255

# 将class vectors转变成binary class metrics
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)

In [ ]:
print('Backend:',keras.backend.image_data_format())

In [ ]:
# 查看当前数据集图像的维度顺序
X_train.shape[1:]

In [ ]:
# Method 1: Switch backend
keras.backend.set_image_data_format('channels_first')
print('Backend:',keras.backend.image_data_format())

# Method 2: reshape
# X_train = X_train.reshape(-1,32,32,3)
# X_train.shape

In [ ]:
def base_model(opt):
    model = Sequential()
    
    # 32个卷积核(feature maps),步长为1，特征图的大小不会改变（周边补充空白），
    model.add(Conv2D(32,(3,3), padding="same", input_shape=X_train.shape[1:]))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(32,(3,3)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    # channel是在前面 (theano后台)
    MaxPooling2D(pool_size=(2, 2), data_format="channels_first")
    model.add(Dropout(0.25))
    
    # 64个卷积核
    model.add(Conv2D(64,(3,3),padding="same"))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(64,(3,3)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    MaxPooling2D(pool_size=(2, 2), data_format="channels_first")
    model.add(Dropout(0.25))
    
    model.add(Flatten())   # Flatten layer
    model.add(Dense(512))  # fully connected layer with 512 units
    model.add(Activation('relu'))
    model.add(Dropout(0.25))
    model.add(Dense(num_classes)) # Fully connected output layer with 10 units
    model.add(Activation('softmax')) # softmax activation function
    
    model.compile(loss='categorical_crossentropy',
                  optimizer=opt,
                  metrics=['accuracy']) # 要优化的是准确率
    return model

In [ ]:
def VGG16(opt):
    model = Sequential()
    
    model.add(Conv2D(64,(3,3),padding="same",input_shape=X_train.shape[1:]))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(64,(3,3)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2,2),strides=2,padding="same"))
    
    model.add(Conv2D(128,(3,3),padding="same"))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(128,(3,3),padding="same"))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2,2),strides=2,padding="same"))
    
    model.add(Conv2D(256,(3,3),padding="same"))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(256,(3,3),padding="same"))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(256,(3,3),padding="same"))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2,2),strides=2,padding="same"))
    
    model.add(Conv2D(512,(3,3),padding="same"))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(512,(3,3),padding="same"))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(512,(3,3),padding="same"))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2,2),strides=2,padding="same"))
    
    model.add(Conv2D(512,(3,3),padding="same"))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(512,(3,3),padding="same"))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(512,(3,3),padding="same"))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2,2),strides=2,padding="same"))
    
    model.add(Flatten())
    model.add(Dense(512))  # fully connected layer with 512 units
    model.add(Activation('relu'))
    model.add(Dropout(0.25))
    model.add(Dense(512))  # fully connected layer with 512 units
    model.add(Activation('relu'))
    model.add(Dropout(0.25))
    model.add(Dense(num_classes)) # Fully connected output layer with 10 units
    model.add(Activation('softmax')) # softmax activation function
    
    model.compile(loss='categorical_crossentropy',
                  optimizer=opt,
                  metrics=['accuracy']) # 要优化的是准确率
    return model
    

In [ ]:
# 初始化 RMSprop 优化器
opt2 = keras.optimizers.RMSprop(lr=0.0001, decay=1e-6)
vgg16 = VGG16(opt2)
vgg16.summary() # 打印网络结构及其内部参数

In [2]:
def VGG19(opt):
     model.add(Conv2D(64,(3,3),padding="same",input_shape=X_train.shape[1:]))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(64,(3,3)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2,2),strides=2,padding="same"))
    
    model.add(Conv2D(128,(3,3),padding="same"))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(128,(3,3),padding="same"))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2,2),strides=2,padding="same"))
    
    model.add(Conv2D(256,(3,3),padding="same"))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(256,(3,3),padding="same"))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(256,(3,3),padding="same"))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2,2),strides=2,padding="same"))
    
    model.add(Conv2D(512,(3,3),padding="same"))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(512,(3,3),padding="same"))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(512,(3,3),padding="same"))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2,2),strides=2,padding="same"))
    
    model.add(Conv2D(512,(3,3),padding="same"))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(512,(3,3),padding="same"))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(512,(3,3),padding="same"))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2,2),strides=2,padding="same"))
    
    
    model.add(Conv2D(512,(3,3),padding="same"))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(512,(3,3),padding="same"))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(512,(3,3),padding="same"))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2,2),strides=2,padding="same"))
    
    model.add(Flatten())
    model.add(Dense(512))  # fully connected layer with 512 units
    model.add(Activation('relu'))
    model.add(Dropout(0.25))
    model.add(Dense(512))  # fully connected layer with 512 units
    model.add(Activation('relu'))
    model.add(Dropout(0.25))
    model.add(Dense(num_classes)) # Fully connected output layer with 10 units
    model.add(Activation('softmax')) # softmax activation function
    
    model.compile(loss='categorical_crossentropy',
                  optimizer=opt,
                  metrics=['accuracy']) # 要优化的是准确率
    return model

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 3)

In [19]:
history = vgg16.fit(X_train, y_train, 
        epochs = 100, batch_size = 32, 
        validation_data=(X_test,y_test), 
        shuffle=True)

Epoch 1/100
1563/1563 [==============================] - 131s 84ms/step - loss: 1.6222 - accuracy: 0.4019 - val_loss: 1.3278 - val_accuracy: 0.5209
Epoch 2/100
1563/1563 [==============================] - 130s 83ms/step - loss: 1.0602 - accuracy: 0.6267 - val_loss: 1.5124 - val_accuracy: 0.5280
Epoch 3/100
1563/1563 [==============================] - 131s 84ms/step - loss: 0.7985 - accuracy: 0.7254 - val_loss: 0.7666 - val_accuracy: 0.7340
Epoch 4/100
1563/1563 [==============================] - 130s 83ms/step - loss: 0.6166 - accuracy: 0.7908 - val_loss: 0.7366 - val_accuracy: 0.7503
Epoch 5/100
1563/1563 [==============================] - 130s 83ms/step - loss: 0.4870 - accuracy: 0.8373 - val_loss: 0.7701 - val_accuracy: 0.7456
Epoch 6/100
1563/1563 [==============================] - 130s 83ms/step - loss: 0.3786 - accuracy: 0.8751 - val_loss: 0.7852 - val_accuracy: 0.7577
Epoch 7/100
1563/1563 [==============================] - 131s 84ms/step - loss: 0.2991 - accuracy: 0.9033 - val_

KeyboardInterrupt: 

In [ ]:
# 初始化 RMSprop 优化器
opt1 = keras.optimizers.RMSprop(lr=0.0001, decay=1e-6)
# 初始化 Adam 优化器
# opt2 = keras.optimizers.Adam(lr=0.0001)

# 用RMSProp训练模型
cnn2 = base_model(opt1)
cnn2.summary() # 打印网络结构及其内部参数

In [ ]:
history = cnn2.fit(X_train, y_train, 
        epochs = 100, batch_size = 32, 
        validation_data=(X_test,y_test), 
        shuffle=True)

In [ ]:
# 对样本进行测试，默认不使用正则化
score2 = cnn2.evaluate(X_test,y_test)
print("损失值为{0:.2f},准确率为{1:.2%}".format(score2[0],score2[1]))

In [ ]:
def plot_loss_and_accuracy(history):
    # Plots for training and testing process: loss and accuracy
 
    plt.figure(0)
    plt.plot(history.history['accuracy'],'r')
    plt.plot(history.history['val_accuracy'],'g')
    plt.xticks(np.arange(0, 101, 20))
    plt.rcParams['figure.figsize'] = (10, 8)
    plt.xlabel("Num of Epochs")
    plt.ylabel("Accuracy")
    plt.title("Training Accuracy vs Validation Accuracy")
    plt.legend(['train','validation'])
     
     
    plt.figure(1)
    plt.plot(history.history['loss'],'r')
    plt.plot(history.history['val_loss'],'g')
    plt.xticks(np.arange(0, 101, 20))
    plt.rcParams['figure.figsize'] = (10, 8)
    plt.xlabel("Num of Epochs")
    plt.ylabel("Loss")
    plt.title("Training Loss vs Validation Loss")
    plt.legend(['train','validation'])
     
     
    plt.show()

In [ ]:
plot_loss_and_accuracy(history)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
Y_pred = cnn2.predict(X_test, verbose=2)
y_pred = np.argmax(Y_pred, axis=1)
 
# for ix in range(10):
#     print(ix, confusion_matrix(np.argmax(y_test,axis=1),y_pred)[ix].sum())
cm = confusion_matrix(np.argmax(y_test,axis=1),y_pred)
print(cm)

# Visualizing of confusion matrix
import seaborn as sn
import pandas  as pd
 
 
df_cm = pd.DataFrame(cm, range(10),range(10))
plt.figure(figsize = (10,7))
sn.set(font_scale=1.4)#for label size
sn.heatmap(df_cm, 
            annot=True,
            annot_kws={"size": 12})# font size
plt.show()

In [ ]:
save_dir = os.path.join('cifar-10-python/', 'saved_models')
model_name = 'keras_cifar10_trained_model.h5'

# =======================
config = cnn2.get_config()
model = cnn2.from_config(config)
cnn2.save(save_dir+model_name)